In [1]:
import cv2
import numpy as np
from roboflow import Roboflow
from ultralytics import YOLO
from PIL import Image
import math
import time
import snap7
import openvino as ov

In [2]:
import openvino as ov

In [ ]:
core = ov.Core()
core.get_available_devices()
# core.read_model("Intel(R) Iris(R) Xe Graphics")
# comp_model = core.compile_model("handle_cover_m2_openvino_model/handle_cover_m2.xml", "GPU")

In [ ]:
cpu_device_name = core.get_property("GPU", "FULL_DEVICE_NAME")

In [ ]:
cpu_device_name

In [ ]:
# TRAINING CUSTOM DATASET WITH PRE- ANNOTATED IMAGES FROM ROBOFLOW

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO

rf = Roboflow(api_key="Om8Ts8Ct9CobGkaiQI86")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(2)
dataset = version.download("yolov8")

model = YOLO("yolov8m.pt")
model.train(data = "data.yaml", epochs = 2)


In [ ]:
# IMAGE OBJECT DETECTION

In [ ]:
model = YOLO("handle_cover_nano_400.pt")
results = model.predict(source = "9002592753X1505D_CE_M2XX101_2_20240604125611_NOK.png", imgsz=400)

result = results[0]
len(result.boxes)
result.boxes
for box in result.boxes:
    label = result.names[box.cls[0].item()]
    cords = [round(x) for x in box.xyxy[0].tolist()]
    prob = box.conf[0].item()
    print("Object type: ", label)
    print("Coordinates: ", cords)
    print("Probability: ", prob)
    print("---")


Image.fromarray(result.plot()[:,:,::-1])
    

In [ ]:
# VIDEO CAPTURE 

In [ ]:
import cv2
import numpy as np
from roboflow import Roboflow
from ultralytics import YOLO
 
prev_frame_time = 0

def fps():
    global prev_frame_time
    new_frame_time = time.time()
    fps =1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    fps = str(int(fps))

    return fps

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))

print("Frame default resolution: (" + str(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) + "; " + str(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) + ")")


if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
# Read until video is completed

frames = 0
total_time = 0

while(True):
  # Capture frame-by-frame
  start_time = time.time()
  frames+=1
  
  ret, frame = cap.read()
  frame= cv2.resize(frame, (100,100))


  if total_time>= 10:
    fps1 = (frames)/total_time
    cv2.putText(frame, fps1, (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 0), 1)

  if ret == True:
      
    

    cv2.imshow('Frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
  end_time = time.time()
  diff = end_time-start_time
  print(diff)
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [ ]:
# VIDEO DETECTION

In [ ]:
# TRACKING WITH YOLO TRACK

model = YOLO('handle_cover_nano_400.pt')

results = model.track(source = "DPMVideo 2024-8-14 9-34-0.mp4", show=True, imgsz = 640, tracker ="bytetrack.yaml")

In [ ]:
cap = cv2.VideoCapture("IMG_0066.mov")

while True:
    success, frame = cap.read()
    frame = cv2.resize(frame, (640, 640)) 
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("w %f, h %f" % (w,h))
    print("width %f, height %f" % (width, height))

    time.sleep(10)

In [ ]:
# TRACKING WITH OPENCV from a video
conf_arr = []

def put_boxes(box, frame):
    global conf_arr
    # bounding box
    confidence = math.ceil(box.conf[0]*100)/100
    conf_arr.append(confidence)
    if confidence >=0.6:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
        #put box in cam
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

        #put text in cam
        org = [x1, y1-5]
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 0.5
        color = (0, 255, 0)
        thickness = 1
        cv2.putText(frame, ("%.2f") % confidence, org, font, fontScale, color, thickness)
    else:
        pass

classNames = ["handle"]

#model = YOLO('handle_cover_nano_400.pt')
openvino_model = YOLO("handle_cover_nano_400_v3_133_openvino_model/")

cap = cv2.VideoCapture("DPMVideo 2024-8-23 15-9-50.mp4")

while True:
    success, frame = cap.read()
    
    results = openvino_model(frame, imgsz=640, iou = 0.1)
    #results = model(frame, stream=True, imgsz = 640, iou = 0.1)
    window_name = 'image'

    # coordinates
    for r in results:
        boxes = r.boxes
        handle_count = 0
        cover_count = 0

        for box in boxes:
            # bounding box
            label = r.names[box.cls[0].item()] 

            if label == "handle":
                handle_count +=1
            elif label == "cover":
                cover_count +=1      
            print("%d %d" % (handle_count, cover_count))

            confidence = math.ceil((box.conf[0]*100))/100
            if confidence >= 0.6:
                put_boxes(box, frame)

    frame = cv2.resize(frame, (1728,972))
    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) == ord('q'):
        break


sum = 0
for i in range(len(conf_arr)):
    sum += conf_arr[i]

if len(conf_arr)!=0:
    average_conf = sum/(len(conf_arr))
else:
    average_conf = 0
print(average_conf)
cap.release()
cv2.destroyAllWindows()

In [ ]:
conf_arr

In [ ]:
print(average_conf)

In [ ]:
# TRACKING WITH OPENCV from webcam

classNames = ["handle"]

model = YOLO('handle_cover_small.pt')

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))

while True:
    start = time.time()
    success, frame = cap.read()
    results = model(frame, stream=True, imgsz = 640, iou = 0.1)
    window_name = 'Webcam'

    for r in results:
        boxes = r.boxes

        for box in boxes:

            # bounding box
            confidence = math.ceil((box.conf[0]*100))/100
            cls = int(box.cls[0])
            if confidence >=0.6:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                #put box in cam
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

                #put text in cam
                org = [x1, y1-5]
                org2 = [x2, y2-5]
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 0.5
                color = (0, 255, 0)
                thickness = 1
                cv2.putText(frame, ("%.2f") % confidence, org, font, fontScale, color, thickness)
            else:
                pass

    frame = cv2.resize(frame, (100,100))
    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) == ord('q'):
        break
    end = time.time()
    print(end-start)
cap.release()
cv2.destroyAllWindows()

In [ ]:
model = YOLO('handle_cover_nano_400_v3_133.pt')
model.export(format = 'openvino', imgsz = 640)

In [ ]:
# TRACKING WITH OPENCV from webcam with OpenVINO

classNames = ["handle"]

model = YOLO('handle_cover_m2.pt')
#model.export(format = 'openvino')
openvino_model = YOLO("handle_cover_m_test_openvino_model/")

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))

while True:

    success, frame = cap.read()
    frame = cv2.resize(frame, (640, 640),)
    #results=model(frame, imgsz=640, iou = 0.1)
    results = openvino_model(frame, imgsz=640)
    window_name = 'Webcam'

    for r in results:
        boxes = r.boxes

        for box in boxes:

            # bounding box
            confidence = math.ceil((box.conf[0]*100))/100
            cls = int(box.cls[0])
            print(cls)
            if confidence >=0.6:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                #put box in cam
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

                #put text in cam
                org = [x1, y1-5]
                org2 = [x2, y2-5]
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 0.5
                color = (0, 255, 0)
                thickness = 1
                cv2.putText(frame, ("%.2f") % confidence, org, font, fontScale, color, thickness)
            else:
                pass

    
    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# TRACKING WITH OPENCV from webcam PLC
import cv2
import numpy as np
from roboflow import Roboflow
from ultralytics import YOLO
from PIL import Image
import math
import time
import snap7

classNames = ["handle"]

model = YOLO('Handle.pt')

cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
#cap.set(cv2.CAP_PROP_FPS, 60)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))

plc = snap7.client.Client()
plc.connect('192.168.1.119', 0 , 1)
bytes = plc.db_read(1,10,1)
test_start = snap7.util.get_bool(bytes, 0, 0)


while True:

    success, frame = cap.read()
    #frame = cv2.resize(frame, (640, 360))
    results = model(frame, stream=True)
    window_name = 'Webcam'

    for r in results:
        boxes = r.boxes

        #region SNAP7
        test_int = plc.db_read(1,8,2)
        snap7.util.set_int(test_int, 0, len(boxes))
        plc.db_write(1, 8, test_int)

        if len(boxes) != 0:
            test_bool = plc.db_read(1,6,1)
            snap7.util.set_bool(test_bool, 0, 0, True)
            plc.db_write(1, 6, test_bool)
        elif len(boxes) == 0:
            test_bool = plc.db_read(1,6,1)
            snap7.util.set_bool(test_bool, 0, 0, False)
            plc.db_write(1, 6, test_bool)    
        #endregion

        for box in boxes:

            # bounding box
            confidence = math.ceil(box.conf[0]*100)/100
            if confidence >=0.6:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                #put box in cam
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

                #put text in cam
                org = [x1, y1-5]
                org2 = [x2, y2-5]
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 0.5
                color = (0, 255, 0)
                thickness = 1
                cv2.putText(frame, ("%.2f") % confidence, org, font, fontScale, color, thickness)
            else:
                pass


    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2, math
from roboflow import Roboflow
from ultralytics import YOLO
import snap7
import os, sys
import multiprocessing
import time

def resource_path(relative):

    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative)


def put_boxes(box, frame):

    # bounding box
    confidence = math.ceil(box.conf[0]*100)/100
    if confidence >=0.6:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
        #put box in cam
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

        #put text in cam
        org = [x1, y1-5]
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 0.5
        color = (0, 255, 0)
        thickness = 1
        cv2.putText(frame, ("%.2f") % confidence, org, font, fontScale, color, thickness)
    else:
        pass

def get_video(source):
    cap = cv2.VideoCapture(source, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))

    return cap
def main():
    
    cap = get_video(0)

    pt = resource_path('handle_cover_small.pt')
    model = YOLO(pt)
    #pt2 =resource_path("handle_cover_m2_openvino_model/")
    #openvino_model = YOLO(pt2)
    plc = snap7.client.Client()

    while True:
                
        success, frame = cap.read()
        if success == False:
            cap.release()
            cap = get_video(0)
            success, frame = cap.read()
        else:
            pass
                        
        if not plc.get_connected():
            try:
                plc.connect('192.168.1.19', 0 , 1)
            except:
                time.sleep(2)
                pass           

        elif plc.get_connected():
            test_start_bytes = plc.db_read(20,2,1)
            test_start = snap7.util.get_bool(test_start_bytes, 0, 0)

            results = model(frame, stream=True, iou = 0.1)
            #results = openvino_model(frame, imgsz = 640, iou = 0.1)
            for r in results:
                boxes = r.boxes
                handle_count = 0
                cover_count = 0
                
                for box in boxes:
                    label = r.names[box.cls[0].item()] 
                    put_boxes(box, frame)
                
                    if label == "handle":
                        handle_count +=1
                    elif label == "cover":
                        cover_count +=1      

                if test_start:

                    recete_bytes = plc.db_read(20,0,2)
                    recete_no = snap7.util.get_int(recete_bytes, 0)   
                    print("Recete: %d" % recete_no)    

                    if recete_no == 2 or recete_no == 8:       

                        test_ok = plc.db_read(20,10,1)

                        if handle_count == 2 and cover_count == 2:                   
                            snap7.util.set_bool(test_ok, 0, 0, True)
                            plc.db_write(20, 10, test_ok)
                            print("TEST COMPLETED")
                        else:
                            snap7.util.set_bool(test_ok, 0, 0, False)
                            plc.db_write(20, 10, test_ok)    
                            
                    elif recete_no == 6:
                        
                        test_ok = plc.db_read(20,10,1)

                        if cover_count == 2 and handle_count == 0:                    
                            snap7.util.set_bool(test_ok, 0, 0, True)
                            plc.db_write(20, 10, test_ok)
                            print("TEST COMPLETED")
                        else:                    
                            snap7.util.set_bool(test_ok, 0, 0, False)
                            plc.db_write(20, 10, test_ok)        

        frame = cv2.resize(frame, (640, 640))            
        cv2.imshow("handle_cover_detection", frame)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    multiprocessing.freeze_support()
    main()